In [1]:
%%writefile example2.txt
first 
second line
the third line
then a forth line

Writing example2.txt


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [4]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2

In [5]:
text_rdd = sc.textFile('example2.txt.txt')

In [6]:
words = text_rdd.map(lambda line: line.split())

In [7]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'forth', 'line']]

In [8]:
text_rdd.collect()

['first ', 'second line', 'the third line', 'then a forth line']

In [10]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'forth',
 'line']

In [11]:
%%writefile service.txt
#EventID  Timestamp      Customer  State   ServiceID.   Amount
201     10/13/2017       100       NY      131          100.00
204     10/18/2017       700       TX      129          450.00
202     10/15/2017       203       CA      121          200.00
206     10/19/2017       202       CA      131          500.00
203     10/17/2017       101       NY      173          750.00
205     10/19/2017       202       TX      121          200.00

Writing service.txt


In [12]:
services = sc.textFile('service.txt')

In [13]:
services.take(2)

['#EventID  Timestamp      Customer  State   ServiceID.   Amount',
 '201     10/13/2017       100       NY      131          100.00']

In [15]:
services.map(lambda line: line.split()).take(3)

[['#EventID', 'Timestamp', 'Customer', 'State', 'ServiceID.', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [25]:
services.map(lambda line: line[1:] if line[0]=='#' else line).collect()

['EventID  Timestamp      Customer  State   ServiceID.   Amount',
 '201     10/13/2017       100       NY      131          100.00',
 '204     10/18/2017       700       TX      129          450.00',
 '202     10/15/2017       203       CA      121          200.00',
 '206     10/19/2017       202       CA      131          500.00',
 '203     10/17/2017       101       NY      173          750.00',
 '205     10/19/2017       202       TX      121          200.00']

In [26]:
clean = services.map(lambda line: line[1:] if line[0]=='#' else line)

In [28]:
clean = clean.map(lambda line: line.split())

In [29]:
clean.collect()

[['EventID', 'Timestamp', 'Customer', 'State', 'ServiceID.', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [31]:
clean.map(lambda lst: (lst[3],lst[-1])).collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [46]:
pairs = clean.map(lambda lst: (lst[3],lst[-1]));

In [47]:
pairs.collect();

In [48]:
rekey = pairs.reduceByKey(lambda amt1,amt2: float(amt1)+float(amt2));

In [50]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [51]:
clean.collect()

[['EventID', 'Timestamp', 'Customer', 'State', 'ServiceID.', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [52]:
#grab (state, action)
step1 = clean.map(lambda lst:(lst[3],lst[-1]))
#reduce by key
step2 = step1.reduceByKey(lambda amt1,amt2: float(amt1)+float(amt2))
#get rid of state, amout titles
step3 = step2.filter(lambda x: not x[0]=='State')
#sort results by amount
step4 = step3.sortBy(lambda stAmount: stAmount[1], ascending =False)
#Action
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [53]:
x = ['ID', 'State', 'Amount']

In [54]:
def func1(lst):
    return lst[-1]

In [55]:
def func2(id_st_amt):
    #unpack values
    (ID,st,amt) = id_st_amt
    return amt

In [56]:
func1(x)

'Amount'

In [57]:
func2(x)

'Amount'